In [1]:
import synapseclient
import synapseutils
from synapseclient import Project, File, Folder
from synapseclient import Schema, Column, Table, Row, RowSet, as_table_columns
import pandas as pd

In [2]:
syn = synapseclient.Synapse()
syn.login()


UPGRADE AVAILABLE

A more recent version of the Synapse Client (3.2.0) is available. Your version (3.1.1) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 3.2.0 release notes

https://python-docs.synapse.org/build/html/news.html

[WARNING] Logging into Synapse via apikeys will be deprecated by early 2024.  If you've used `rememberMe=True` in the past, you may also be getting this message, please delete your ~/.synapseCache/.session file. This message will disappear if you use a Synapse Personal Access Token to login.


Welcome, Victor Baham!



### Basic idea:

1. Extract `value` column from `table_AD_model`
2. Loop through each cell in `table_AD_backend`:

    a. In cases where the term does not exist in `value`, suggest 3 alternatives based on Levenshtein distance
    
    b. Otherwise, admit defeat
    

### Actual implementation:

* Access individual cells using `df.iloc[row, col]`
* Highlight changed cells

In [3]:
schema_AD_backend = syn.get('syn11346063')
results_AD_backend = syn.tableQuery(f"SELECT * FROM {schema_AD_backend.id}")  
table_AD_backend = pd.read_csv(results_AD_backend.filepath)

table_AD_backend

[WARNING] /var/folders/lg/w3yckgy936lc0r6yw790hjs00000gr/T/ipykernel_5286/736888456.py:3: DtypeWarning: Columns (14,15,18,19,22,23,24,25,28,29,30,31,41,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  table_AD_backend = pd.read_csv(results_AD_backend.filepath)



,ROW_ID,ROW_VERSION,ROW_ETAG,id,name,study,dataType,assay,organ,tissue,...,benefactorId,projectId,modifiedOn,modifiedBy,dataFileHandleId,metaboliteType,chromosome,modelSystemType,libraryPrep,dataFileSizeBytes
0,2426151,3,60e64259-515c-4609-aaa5-7c2d8ba82dc4,syn2426151,chr1.chop.dosage.gz,"[""ROSMAP""]","[""genomicVariants""]","[""snpArray""]",NaN,NaN,...,syn5550382,syn2580853,1634580268379,3420438,75430845,NaN,NaN,NaN,NaN,6.431568e+08
1,2426152,3,9844ca5a-a422-40ec-8eab-3168769fd309,syn2426152,chr2.chop.dosage.gz,"[""ROSMAP""]","[""genomicVariants""]","[""snpArray""]",NaN,NaN,...,syn5550382,syn2580853,1634580268426,3420438,75430846,NaN,NaN,NaN,NaN,6.977947e+08
2,2426153,3,220e4abd-75d5-4378-990e-2497e76e113a,syn2426153,chr4.chop.dosage.gz,"[""ROSMAP""]","[""genomicVariants""]","[""snpArray""]",NaN,NaN,...,syn5550382,syn2580853,1634580268473,3420438,75430848,NaN,NaN,NaN,NaN,5.863906e+08
3,2426154,3,ca15e26b-8672-4b1f-8aae-9b55852186eb,syn2426154,chr8.chop.dosage.gz,"[""ROSMAP""]","[""genomicVariants""]","[""snpArray""]",NaN,NaN,...,syn5550382,syn2580853,1634580268524,3420438,75430847,NaN,NaN,NaN,NaN,4.498432e+08
4,2426155,3,dffa1f8a-6b1d-4375-bf9f-397a1fc3a9f0,syn2426155,chr9.chop.dosage.gz,"[""ROSMAP""]","[""genomicVariants""]","[""snpArray""]",NaN,NaN,...,syn5550382,syn2580853,1634580268573,3420438,75430849,NaN,NaN,NaN,NaN,3.599844e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239657,53188016,1,8b9e0260-228a-40f6-a66d-483ebae8dfae,syn53188016,HL61.r2.fq.gz,NaN,NaN,NaN,NaN,NaN,...,syn52383692,syn2580853,1703303078950,3353934,131779500,NaN,NaN,NaN,NaN,2.449366e+09
239658,53188860,1,7035ddbc-2ecf-42d2-b6fc-5379b5f001cc,syn53188860,HL66.r1.fq.gz,NaN,NaN,NaN,NaN,NaN,...,syn52383692,syn2580853,1703307359936,3353934,131780076,NaN,NaN,NaN,NaN,2.911883e+09
239659,53189079,1,ef56d915-33e2-4415-8ddf-61b24bb783de,syn53189079,HL66.r2.fq.gz,NaN,NaN,NaN,NaN,NaN,...,syn52383692,syn2580853,1703308481208,3353934,131780193,NaN,NaN,NaN,NaN,3.048309e+09
239660,53189311,1,e6fc774c-642f-4b1d-bcfc-8da412365129,syn53189311,HL67.r1.fq.gz,NaN,NaN,NaN,NaN,NaN,...,syn52383692,syn2580853,1703309528121,3353934,131780260,NaN,NaN,NaN,NaN,2.759696e+09


In [4]:
schema_AD_model = syn.get('syn53010627')
results_AD_model = syn.tableQuery(f"SELECT * FROM {schema_AD_model.id}")
table_AD_model = pd.read_csv(results_AD_model.filepath)

table_AD_model

,ROW_ID,ROW_VERSION,key,description,columnType,maximumSize,value,valueDescription,source,module
0,2,2,ageAssessment,Age of clinical or behavioral assessment.,number,NaN,NaN,NaN,sage.annotations-experimentalData.ageAssessmen...,experimentalData
1,3,2,ageAssessmentUnits,Age of assessment units.,string,NaN,days,"Age measured in days, a period of 24 hours.",sage.annotations-experimentalData.ageAssessmen...,experimentalData
2,4,2,ageAssessmentUnits,Age of assessment units.,string,NaN,gestational weeks,Gestational age (written with both weeks and d...,sage.annotations-experimentalData.ageAssessmen...,experimentalData
3,5,2,ageAssessmentUnits,Age of assessment units.,string,NaN,months,Age measured in calendar months (approximately...,sage.annotations-experimentalData.ageAssessmen...,experimentalData
4,6,2,ageAssessmentUnits,Age of assessment units.,string,NaN,PCW,Post-Conception Weeks,sage.annotations-experimentalData.ageAssessmen...,experimentalData
...,...,...,...,...,...,...,...,...,...,...
2551,2553,3,individual_animal_metadata_template,SysBio individual animal metadata template schema,NaN,NaN,NaN,NaN,sysbio.metadataTemplates-individual.animal,template
2552,2554,3,individual_animal_modelad_metadata_template,individual animal metadata template schema for...,NaN,NaN,NaN,NaN,sysbio.metadataTemplates-modelad.individual.an...,template
2553,2555,3,individual_human_metadata_template,SysBio AD individual human metadata template s...,NaN,NaN,NaN,NaN,sysbio.metadataTemplates-ad.individual.human,template
2554,2556,3,ATP6V1A_kd,The ATP6V1A knockdown in hIPSC neurons (ATP6V...,string,NaN,NaN,NaN,https://www.synapse.org/#!Synapse:syn52201354,ADKP


In [6]:
ctrl_vals = table_AD_model['value'].astype(str).tolist()
ctrl_vals

['nan',
 'days',
 'gestational weeks',
 'months',
 'PCW',
 'weeks',
 'years',
 'nan',
 'nan',
 'days',
 'gestational weeks',
 'months',
 'PCW',
 'weeks',
 'years',
 'ANOVA',
 'assessment',
 'batch effect correction',
 'chromatin accessible quantitative trait loci detection',
 'Clustering',
 'comparison',
 'Copy number estimation',
 'correlation',
 'covariate specification',
 'data mining',
 'data normalization',
 'de-novo assembly',
 'differential expression',
 'differential methylation',
 'differential network analysis',
 'DNA methylation imputation',
 'dose response study',
 'Enrichment analysis',
 'EWAS',
 'expression quantitative trait loci detection',
 'Gene expression comparison',
 'Genome-Wide Association',
 'genome-wide pleiotropy',
 'genotypeAncestry',
 'genotype imputation',
 'hidden confounder detection',
 'manifold learning',
 'Mendelian randomization analysis',
 'methylation quantitative trait loci detection',
 'network analysis',
 'network driver detection',
 'outlier det

In [13]:
i = 0
j = 0

chr_rmv = ['[', ']', '"']

for val in table_AD_backend['study'].astype(str):
    for char in chr_rmv:
        val = val.replace(char, "")
        
    